## Tensorflow Serving Version 1.13 Example

Train and export a simple Softmax Regression TensorFlow model.
The model is from the TensorFlow "MNIST For ML Beginner" tutorial. This program
simply follows all its training instructions, and uses TensorFlow SavedModel to
export the trained model with proper signatures that can be loaded by standard
tensorflow_model_server.
<br>
Usage: mnist_saved_model.py [--training_iteration=x] [--model_version=y] export_dir

In [1]:
from __future__ import print_function

import os
import sys
import pickle
import numpy as np
import tensorflow as tf
from tensorflow.contrib.session_bundle import exporter

In [2]:
export_path_base = "./model"
training_iterations = 1000
model_version = 1

In [3]:
sess = tf.InteractiveSession()
 
x = tf.placeholder('float', shape=[None, 3], name = 'x')
y_ = tf.placeholder('float', shape=[None, 1], name = 'y')

w = tf.get_variable(name = 'w', shape = [3,1], initializer = tf.truncated_normal_initializer)
b = tf.get_variable(name = 'b', shape = [1], initializer = tf.zeros_initializer)
 
sess.run(tf.global_variables_initializer())
 
y = tf.matmul(x, w) + b
 
ms_loss = tf.reduce_mean((y - y_)**2)
 
train_step = tf.train.GradientDescentOptimizer(0.005).minimize(ms_loss)

train_x = np.random.randn(1000, 3)
# let the model learn the equation of y = x1 * 1 + x2 * 2 + x3 * 3
train_y = np.sum(train_x * np.array([1,2,3]) + np.random.randn(1000, 3) / 100, axis = 1).reshape(-1, 1)

train_loss = []

for it in range(training_iterations):
    loss, _ = sess.run([ms_loss, train_step], feed_dict={x: train_x, y_: train_y})
    train_loss.append(loss)
    
print('Training error %g' % loss)
print('Done training!')


export_path = os.path.join(
    tf.compat.as_bytes(export_path_base),
    tf.compat.as_bytes(str(model_version))
)

print('Exporting trained model to', export_path)
builder = tf.saved_model.builder.SavedModelBuilder(export_path)
 
tensor_info_x = tf.saved_model.utils.build_tensor_info(x)
tensor_info_y = tf.saved_model.utils.build_tensor_info(y)
 
prediction_signature = (
    tf.saved_model.signature_def_utils.build_signature_def(
        inputs={'input': tensor_info_x},
        outputs={'output': tensor_info_y},
        method_name=tf.saved_model.signature_constants.PREDICT_METHOD_NAME
    )
)

legacy_init_op = tf.group(tf.tables_initializer(), name='legacy_init_op')

builder.add_meta_graph_and_variables(
    sess, [tf.saved_model.tag_constants.SERVING],
    signature_def_map={
        'prediction':
        prediction_signature,
    },
    legacy_init_op=legacy_init_op
)
 
builder.save()
 
print('Done exporting!')
print('Input-Tensor-Shap: ', tensor_info_x)

Instructions for updating:
Colocations handled automatically by placer.
Training error 0.00029757
Done training!
Exporting trained model to b'./model/1'
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
Instructions for updating:
Pass your op to the equivalent parameter main_op instead.
INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: ./model/1/saved_model.pb
Done exporting!
Input-Tensor-Shap:  name: "x:0"
dtype: DT_FLOAT
tensor_shape {
  dim {
    size: -1
  }
  dim {
    size: 3
  }
}



### Start Tensorflow Serving (don't specify model version)

docker run -p 8501:8501 --mount type=bind,source=/home/bai/PycharmProjects/tensorflow_for_enginner/11_tf_serving/model,target=/models/my_model -e MODEL_NAME=my_model -t tensorflow/serving

curl -X POST http://localhost:8501/v1/models/my_model:predict -d '{"signature_name":"prediction","instances":[{"input":[1,2,3]}]}'

In [5]:
train_x = np.random.randn(2, 1)
print(train_x, train_x.shape)

[[ 0.1680071 ]
 [-1.79930324]] (2, 1)
